In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [6]:
# sort classes
classes = sorted(list(set(classes)))

In [7]:
# documents = combination between patterns and intents
print (len(documents), "documents")

95 documents


In [8]:
# classes = intents
print (len(classes), "classes", classes)

16 classes ['courtesy_greting', 'floxus', 'goodbye', 'greeting', 'help', 'intuitive_challenges', 'live_challenges', 'name', 'options', 'practical_projects', 'profile_verification', 'real_name', 'shutup', 'thanks', 'unbound', 'understand']


In [9]:
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

107 unique lemmatized words ["'m", "'s", ',', 'a', 'afternoon', 'am', 'any', 'anyone', 'are', 'awesome', 'be', 'better', 'bye', 'call', 'can', 'challenge', 'chatting', 'complete', 'comprendo', 'could', 'day', 'do', 'doe', 'doing', 'enough', 'floxus', 'for', 'friend', 'get', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'heya', 'hi', 'hola', 'hope', 'how', 'hya', 'i', 'internshala', 'intuitive', 'is', 'know', 'later', 'like', 'list', 'live', 'me', 'mean', 'morning', 'my', 'name', 'need', 'next', 'nice', 'night', 'of', 'offered', 'on', 'opened', 'others', 'platform', 'please', 'practical', 'profile', 'project', 'provide', 'quiet', 'real', 'saying', 'see', 'shhh', 'shut', 'start', 'stop', 'stuck', 'support', 'talking', 'tell', 'than', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'udemy', 'unbound', 'understand', 'up', 'well', 'what', 'when', 'where', 'who', 'will', 'with', 'work', 'working', 'you', 'your']


In [10]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [11]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [12]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [13]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-13-9ffac2c3f218>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [17]:
len(train_y[0])

16

In [14]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [15]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [16]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h1', hist)

print("model created")


Epoch 1/200
19/19 [==============================] - 0s 2ms/step - loss: 2.7938 - accuracy: 0.0632
Epoch 2/200
19/19 [==============================] - 0s 948us/step - loss: 2.6868 - accuracy: 0.1368
Epoch 3/200
19/19 [==============================] - 0s 917us/step - loss: 2.5946 - accuracy: 0.1895
Epoch 4/200
19/19 [==============================] - 0s 535us/step - loss: 2.4204 - accuracy: 0.2947
Epoch 5/200
19/19 [==============================] - 0s 674us/step - loss: 2.2974 - accuracy: 0.3474
Epoch 6/200
19/19 [==============================] - 0s 968us/step - loss: 2.1402 - accuracy: 0.4421
Epoch 7/200
19/19 [==============================] - 0s 947us/step - loss: 1.9843 - accuracy: 0.4737
Epoch 8/200
19/19 [==============================] - 0s 1000us/step - loss: 1.9595 - accuracy: 0.4421
Epoch 9/200
19/19 [==============================] - 0s 1ms/step - loss: 1.6822 - accuracy: 0.5263
Epoch 10/200
19/19 [==============================] - 0s 943us/step - loss: 1.6046 - accuracy:

19/19 [==============================] - 0s 1ms/step - loss: 0.0642 - accuracy: 0.9895
Epoch 162/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0295 - accuracy: 0.9895
Epoch 163/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0777 - accuracy: 0.9684
Epoch 164/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0807 - accuracy: 0.9579
Epoch 165/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0442 - accuracy: 0.9789
Epoch 166/200
19/19 [==============================] - 0s 803us/step - loss: 0.0408 - accuracy: 1.0000
Epoch 167/200
19/19 [==============================] - 0s 1ms/step - loss: 0.1027 - accuracy: 0.9579
Epoch 168/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0658 - accuracy: 0.9895
Epoch 169/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0370 - accuracy: 0.9895
Epoch 170/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0474 - accuracy: 0.9895
Ep